In [1]:
#imports
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import skimage.io
import datetime
import random
from scipy.signal import spectrogram

#PyTorch stuff
import torch
import torchvision as torchv
import torch.nn as nn 
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision.datasets import ImageFolder
import gc
from tensorflow.keras.preprocessing.sequence import pad_sequences

# File paths
TESTDIR = './../Data/ToyotaLc/'
TESTPATH = 'Lock/'
TESTSIG = 'keyfob_signal2023Y1022163258.raw'
TESTOUT = './../Data/MelSpecV1/Hyundai/'

IMG_SIZE=120
IMG_L = 120

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ('Lock', 'Unlock','None')
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:1024"
torch.cuda.set_per_process_memory_fraction(0.9)

2023-12-15 17:08:56.346234: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 17:08:56.367009: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 17:08:56.394963: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 17:08:56.394979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 17:08:56.396973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
signals_path = './../Data/Hyundai/'

In [3]:
# List categories (e.g., Category1, Category2)
categories = ['Lock', 'Unlock','None']
X = []
y = []

for category in categories:
    input_category_directory = os.path.join(signals_path, category)

    # List all files in the input category directory
    file_list = [f for f in os.listdir(input_category_directory) if f.endswith('.raw')]

    # Loop through the RF signal files in the category
    for file_name in file_list:
        # Construct the full file paths
        input_file_path = os.path.join(input_category_directory, file_name)

        # Read the RF signal data from the input file and convert it to a NumPy array
        with open(input_file_path, 'rb') as file:
            raw_data = file.read()
            rf_signal = np.frombuffer(raw_data, dtype=np.int32).astype(np.float32)
        X.append(rf_signal)
        y.append(category)

In [4]:
len(X)

312

In [5]:
len(y)

312

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=43,
)

In [7]:
X_train[200].shape

(15794176,)

In [8]:
y_train[200]

'Lock'

In [9]:
# Pad sequences for both training and testing sets
maxlen = 15000000  # Adjust maxlen as needed
X_train_padded = pad_sequences(X_train, dtype='float32', padding='post', truncating='post', maxlen=maxlen)
X_test_padded = pad_sequences(X_test, dtype='float32', padding='post', truncating='post', maxlen=maxlen)

# Convert the result to NumPy arrays if needed
X_train_padded = np.array(X_train_padded)
X_test_padded = np.array(X_test_padded)

KeyboardInterrupt: 

In [ ]:
X_train_padded.shape

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Take the output from the last time step
        return out

In [ ]:
input_size = 15000000  # Number of features in the input
hidden_size = 20  # Number of hidden units in the RNN
output_size = 3  # Number of output units
seq_length = 10000  # Length of input sequences

In [ ]:
# Create an instance of the model
model = SimpleRNN(input_size, hidden_size, output_size)

# Define a loss function and an optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 1000

for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_padded)
    loss = criterion(outputs, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')